<a href="https://colab.research.google.com/github/abequette/CS167Fall22Code/blob/main/Day05_Notes_kNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 5 Notes: k Nearest Neighbors
The following code includes the steps necessary to build a k nearest neighbors model. We'll go through each step, and then I'll ask you to pull these steps together into a function that you can pass a new observation of a flower in to and it will output the species the model predicts that flower is.

In [1]:
import pandas
from google.colab import drive
drive.mount('/content/drive')
#go through authentication step

Mounted at /content/drive


In [2]:
#import the data and take a look at it
iris_data = pandas.read_csv('/content/drive/MyDrive/CS167/irisData.csv')
iris_data.head()

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
#compare a series of booleans vs. a dataframe with rows from the boolean values
iris_data['sepal length'] < 5.0

In [ ]:
iris_data[iris_data['sepal length'] < 5.0]

In [ ]:
# Calculate the distance
import numpy
#adding a new column
iris_data['distance_to_new'] = numpy.sqrt(
    (5.1-iris_data['petal length'])**2
    +(7.2-iris_data['sepal length'])**2
    +(1.5-iris_data['petal width'])**2
    +(2.5-iris_data['sepal width'])**2)
iris_data

In [ ]:
# Sort the data and display it
sorted_data = iris_data.sort_values(['distance_to_new']) 
sorted_data #display it

In [ ]:
# give the top 5 closest neighbors
sorted_data.iloc[0:5]

In [ ]:
# give the most commonly occuring species of the 5 closest neighbors
sorted_data.iloc[0:5]['species'].mode()
#sorted_data.iloc[0:5]['species'].mode()[0] #return just the string

## Exercise:
Now it's your turn. Use the code snippets from above to build out a kNN function that takes in `specimen`, `data`, and `k` and prints out the prediction. 

In [ ]:
def knn(specimen, training_data, k):
  # your code here


  

In [ ]:
# Use the code below to build a dictionary of values and test your new function:
unknown_iris = {}
unknown_iris['petal length'] = 5.1
unknown_iris['sepal length'] = 7.2
unknown_iris['petal width'] = 1.5
unknown_iris['sepal width'] = 2.5
# 5-NN 
knn(unknown_iris,iris_data,5)

## Exercise #2:
Now it's your turn. Use the code snippets from above to build out a *weighted* kNN function that takes in `specimen`, `training_data`, and `k` and outputs the prediction. 

In [ ]:
import numpy

# weighted kNN
def w_knn(specimen, training_data, k):
  # calculate the distance
  
  # calculate the weights (remember, the weights are 1/d^2)

  # find the closest k

  # use groupby to sum the weights of each species in the closest k
 
  # return the class that has the larger sum of weights



In [ ]:
unknown_iris = {}
unknown_iris['petal length'] = 5.1
unknown_iris['sepal length'] = 7.2
unknown_iris['petal width'] = 1.5
unknown_iris['sepal width'] = 2.5
# 5-NN 
print(w_knn(unknown_iris,iris_data,5))
